In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/orders/2019.csv")
# df now is a Spark DataFrame containing CSV data from "Files/orders/2019.csv".
display(df)


Este código lee un archivo CSV llamado 2019.csv desde la carpeta Files/orders/ y lo carga como un DataFrame de Spark (una tabla en memoria).

    .format("csv"): dice que el archivo es formato CSV.

    .option("header", "true"): indica que la primera fila del CSV tiene los nombres de las columnas.

    .load(...): carga el archivo.

Luego, con display(df) puedes ver los datos en forma de tabla.

💡 En resumen: Carga un archivo CSV como tabla en Spark y muestra su contenido.

In [ ]:
from pyspark.sql.types import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])

df = spark.read.format("csv").schema(orderSchema).load("Files/orders/2019.csv")

display(df)


El código define un esquema personalizado (con nombres y tipos de columnas) y luego lee un archivo CSV (2019.csv) usando ese esquema, asumiendo que el archivo no tiene encabezado. Finalmente, muestra el contenido como una tabla con display(df).

Es código anterior solo afecta al csv generado para el año 2019, 
df = spark.read.format("csv").schema(orderSchema).load("Files/orders/*.csv")
Añadiendo esa linea de código, 

In [ ]:
customers = df['CustomerName', 'Email']

print(customers.count())
print(customers.distinct().count())

display(customers.distinct())

# customers = df.select("CustomerName", "Email")
# Obtiene el mismo resultado.


In [ ]:
customers = df.select("CustomerName", "Email").where(df['Item']=='Road-250 Red, 52')
print(customers.count())
print(customers.distinct().count())

display(customers.distinct())


In [ ]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()

display(productSales)


In [ ]:
from pyspark.sql.functions import *

yearlySales = df.select(year(col("OrderDate")).alias("Year")).groupBy("Year").count().orderBy("Year")

display(yearlySales)


El código cuenta cuántas órdenes (ventas) hubo por año a partir de la columna OrderDate del DataFrame df. Primero, extrae solo el año de cada fecha con year(col("OrderDate")) y lo renombra como "Year". Luego, agrupa todas las filas por ese año con .groupBy("Year"), y cuenta cuántas filas (ventas) hay en cada grupo usando .count(). Después, ordena los resultados por año con .orderBy("Year"), y finalmente, con display(yearlySales) muestra una tabla con dos columnas: el año (Year) y cuántas órdenes hubo ese año (count).

Es como decir: “Muéstrame cuántas ventas se hicieron cada año, ordenadas cronológicamente.”

In [ ]:
from pyspark.sql.functions import *

# Create Year and Month columns
transformed_df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Create the new FirstName and LastName fields
transformed_df = transformed_df.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Filter and reorder columns
transformed_df = transformed_df["SalesOrderNumber", "SalesOrderLineNumber", "OrderDate", "Year", "Month", "FirstName", "LastName", "Email", "Item", "Quantity", "UnitPrice", "Tax"]

# Display the first five orders
display(transformed_df.limit(5))


Este código transforma un DataFrame df que contiene información de órdenes de venta para hacerlo más útil y organizado.

    Se crean nuevas columnas de fecha:
    Usando la columna OrderDate, se crean dos columnas nuevas llamadas Year (año) y Month (mes) con .withColumn(...) usando las funciones year() y month().

    Se separa el nombre completo del cliente:
    A partir de la columna CustomerName (por ejemplo, "Ana García"), se separan el primer nombre (FirstName) y el apellido (LastName) usando split(...). El primer espacio separa el nombre y el apellido. Por ejemplo:

        "Carlos Rivera" → FirstName = "Carlos", LastName = "Rivera"

    Se reorganizan las columnas:
    Se eligen y ordenan solo las columnas que se quieren mostrar: número de orden, fecha, nombre del cliente, producto, precio, impuestos, etc.

    Se muestran las primeras 5 órdenes:
    Con transformed_df.limit(5) se seleccionan solo las primeras 5 filas, y display(...) muestra el resultado en una tabla.

🧾 Resultado final:

Una tabla con 5 filas (órdenes) que muestra columnas limpias y organizadas como:

        SalesOrderNumber	SalesOrderLineNumber	OrderDate	Year	Month	FirstName	LastName	Email	Item



In [ ]:
transformed_df.write.mode("overwrite").parquet('Files/transformed_data/orders')

print ("Transformed data saved!")


In [ ]:
orders_df = spark.read.format("parquet").load("Files/transformed_data/orders")
display(orders_df)


In [ ]:
orders_df.write.partitionBy("Year","Month").mode("overwrite").parquet("Files/partitioned_data")

print ("Transformed data saved!")


In [ ]:
orders_2021_df = spark.read.format("parquet").load("Files/partitioned_data/Year=2021/Month=*")

display(orders_2021_df)


# TRABAJAR CON SQL

In [ ]:
# Create a new table
df.write.format("delta").saveAsTable("salesorders")

# Get the table description
spark.sql("DESCRIBE EXTENDED salesorders").show(truncate=False)


In [ ]:
%%sql
SELECT YEAR(OrderDate) AS OrderYear,
       SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue
FROM salesorders
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear;


# MATPLOTLIB

In [ ]:
sqlQuery = "SELECT CAST(YEAR(OrderDate) AS CHAR(4)) AS OrderYear, \
                SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue, \
                COUNT(DISTINCT SalesOrderNumber) AS YearlyCounts \
            FROM salesorders \
            GROUP BY CAST(YEAR(OrderDate) AS CHAR(4)) \
            ORDER BY OrderYear"
df_spark = spark.sql(sqlQuery)
df_spark.show()


In [ ]:
from matplotlib import pyplot as plt

# matplotlib requires a Pandas dataframe, not a Spark one
df_sales = df_spark.toPandas()

# Create a bar plot of revenue by year
plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'])

# Display the plot
plt.show()


In [ ]:
from matplotlib import pyplot as plt

# Clear the plot area
plt.clf()

# Create a bar plot of revenue by year
plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')

# Customize the chart
plt.title('Revenue by Year')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.xticks(rotation=45)

# Show the figure
plt.show()


In [ ]:
from matplotlib import pyplot as plt

# Clear the plot area
plt.clf()

# Create a Figure
fig = plt.figure(figsize=(8,3))

# Create a bar plot of revenue by year
plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')

# Customize the chart
plt.title('Revenue by Year')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.xticks(rotation=45)

# Show the figure
plt.show()


In [ ]:
from matplotlib import pyplot as plt

# Clear the plot area
plt.clf()

# Create a figure for 2 subplots (1 row, 2 columns)
fig, ax = plt.subplots(1, 2, figsize = (10,4))

# Create a bar plot of revenue by year on the first axis
ax[0].bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')
ax[0].set_title('Revenue by Year')

# Create a pie chart of yearly order counts on the second axis
ax[1].pie(df_sales['YearlyCounts'])
ax[1].set_title('Orders per Year')
ax[1].legend(df_sales['OrderYear'])

# Add a title to the Figure
fig.suptitle('Sales Data')

# Show the figure
plt.show()


# SEABORN

In [ ]:
import seaborn as sns

# Clear the plot area
plt.clf()

# Create a bar chart
ax = sns.barplot(x="OrderYear", y="GrossRevenue", data=df_sales)

plt.show()


In [ ]:
import seaborn as sns

# Clear the plot area
plt.clf()

# Set the visual theme for seaborn
sns.set_theme(style="whitegrid")

# Create a bar chart
ax = sns.barplot(x="OrderYear", y="GrossRevenue", data=df_sales)

plt.show()


In [ ]:
import seaborn as sns

# Clear the plot area
plt.clf()

# Create a line chart
ax = sns.lineplot(x="OrderYear", y="GrossRevenue", data=df_sales)

plt.show()
